In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [36]:
df = pd.read_csv('claims_sample_data.csv')
df.head(5)

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT
0,201801,AncillaryFFS,NaN,Payer F,4281
1,201801,AncillaryFFS,NaN,Payer H,2221
2,201801,AncillaryFFS,NaN,Payer O,3937
3,201801,AncillaryFFS,NaN,Payer W,268
4,201801,AncillaryFFS,ACH,Payer W,151


# General info

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52152 entries, 0 to 52151
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   MONTH             52152 non-null  int64 
 1   SERVICE_CATEGORY  52152 non-null  object
 2   CLAIM_SPECIALTY   51901 non-null  object
 3   PAYER             52152 non-null  object
 4   PAID_AMOUNT       52152 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


It seems that CLAIM_SPECIALTY has null values

In [38]:
df.describe()

,MONTH,PAID_AMOUNT
count,52152.000000,5.215200e+04
mean,201885.029567,3.708707e+03
std,72.867005,3.449842e+04
min,201801.000000,-4.223000e+04
25%,201808.000000,1.940000e+02
50%,201903.000000,5.860000e+02
75%,201911.000000,1.465000e+03
max,202007.000000,2.268611e+06


# Check Duplicates

In [39]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

number of duplicate rows:  (0, 5)


In [40]:
duplicate_rows_df.head(5)

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT


# Check NULLs

In [41]:
print(df.isnull().sum())

MONTH                 0
SERVICE_CATEGORY      0
CLAIM_SPECIALTY     251
PAYER                 0
PAID_AMOUNT           0
dtype: int64


We have null claim specialties

## Fill nas

In [42]:
df = df.fillna('null_specialty')

In [43]:
print("Unique Claim Specialty: ", len(df['CLAIM_SPECIALTY'].unique()))

Unique Claim Specialty:  906


# Check values

In [44]:
df['SERVICE_CATEGORY'].unique()

array(['AncillaryFFS', 'ASCServices', 'ERServices', 'InpatientServices',
       'OutpatientServices', 'PCPEncounter', 'PCPFFS', 'SNFServices',
       'SpecialistFFS', 'SpecialistsFFS'], dtype=object)

In [45]:
print("Unique Service Category: ", len(df['SERVICE_CATEGORY'].unique()))

Unique Service Category:  10


In [46]:
df['PAYER'].unique()

array(['Payer F', 'Payer H', 'Payer O', 'Payer W', 'Payer UN', 'Payer CA',
       'Payer CO', 'Payer S', 'Payer B', 'Payer UL'], dtype=object)

In [47]:
print("Unique Payers number: ", len(df['PAYER'].unique()))

Unique Payers number:  10


In [48]:
df['MONTH'].unique()

array([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808,
       201809, 201810, 201811, 201812, 201900, 201901, 201902, 201903,
       201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911,
       201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007])

In [49]:
print("Unique Months: ", len(df['MONTH'].unique()))

Unique Months:  32


In [50]:
sorted(df['CLAIM_SPECIALTY'].unique())

['ABULATORY SURGICAL CENTER',
 'ACCUP',
 'ACH',
 'ACUPU',
 'ACUPUNCTURIST',
 'ACUTE CARE HOSPITAL',
 'ACUTE SHORT TERM HOSPITAL',
 'ADDMED',
 'ADDPSY',
 'ADLMED',
 'ADVANCED HEART FAILURE AND TRANSPLANT CARDIOLOGY',
 'ADVANCED REGISTERED NURSE PRACTITIONER',
 'ADVANCED RN PRACT',
 'AHFTC',
 'ALLERGY & IMMUNOLOGY',
 'ALLERGY / IMMUNOLOGY',
 'ALLERGY/IMMUNOLOGY',
 'AMBULANCE',
 'AMBULANCE SERVICE',
 'AMBULATORY SURGERY CENTER',
 'AMBULATORY SURGICAL CENTER',
 'AMBULATORY SURGICAL FACILITY',
 'ANALYTICAL LABS',
 'ANAST',
 'ANATOM',
 'ANATOMIC AND CLINICAL PATHOLOGY',
 'ANATOMIC PATHOLOGY',
 'ANES ASSIST',
 'ANESTHESIA',
 'ANESTHESIOLOGIST',
 'ANESTHESIOLOGY',
 'APRN',
 'ARNP',
 'ASC',
 'AUDIO',
 'AUDIOLOGIST',
 'AUDIOLOGY',
 'AUDIOMETRY',
 'Adolescent Medicine',
 'Advanced Registered Nurse Prac',
 'Agencies',
 'Allergy',
 'Allergy / Immunology',
 'Allergy/Immunology',
 'Ambulance',
 'Ambulance - Emergency Land',
 'Ambulance - Land',
 'Ambulance Service Provider',
 'Ambulatory Health Care'

- similar lower and uppercase specialties <br>
- \t confusing symbols <br>
- claim specialties with very similar names like 'EMERGENCY MEDICINE, GEN PRACTICE' and 'EMERGENCY MEDICINE, GENERAL PRACTICE'

## Group similar specialties

In [51]:
df['CLAIM_SPECIALTY_formatted'] = df['CLAIM_SPECIALTY'].apply(lambda x: x.lower())

In [52]:
df['CLAIM_SPECIALTY_formatted'] = df['CLAIM_SPECIALTY_formatted'].apply(lambda x: x.replace('\t', ''))

In [53]:
print("Unique Claim Specialty: ", len(df['CLAIM_SPECIALTY_formatted'].unique()))

Unique Claim Specialty:  786


In [54]:
claims = df['CLAIM_SPECIALTY_formatted'].unique()

## Find very similar claim specialty groups

In [55]:
from collections import Counter

stripJunk = str.maketrans("","","- ")

def getRatio(a,b):
    a = a.lower().translate(stripJunk)
    b = b.lower().translate(stripJunk)
    total  = len(a)+len(b)
    counts = (Counter(a)-Counter(b))+(Counter(b)-Counter(a))
    return 100 - 100 * sum(counts.values()) / total

In [56]:
%time
data = claims

treshold     = 90
minGroupSize = 1

from itertools import combinations

paired = { c:{c} for c in data }
for a,b in combinations(data,2):
    if getRatio(a,b) < treshold: continue
    paired[a].add(b)
    paired[b].add(a)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [57]:
paired

{'null_specialty': {'null_specialty'},
 'ach': {'ach'},
 'advanced registered nurse prac': {'advanced registered nurse prac'},
 'advanced rn pract': {'advanced rn pract'},
 'ambulance': {'ambulance'},
 'ambulance service': {'ambulance service'},
 'ambulatory surgical center': {'abulatory surgical center',
  'ambulatory surgical center',
  'ambulatory surgical centers'},
 'ambulatory transportation services': {'ambulatory transportation services'},
 'anatom': {'anatom'},
 'anesthesiology': {'anesthesiology'},
 'arnp': {'aprn', 'arnp'},
 'behavioral health counseling': {'behavioral health counseling'},
 'card electrophysiolo': {'card electrophysiolo'},
 'cardiac electrophysiology': {'cardiac electrophysiology'},
 'cardiology': {'cardiology', 'radiology'},
 'cardiology/cardiovascular disease': {'cardiology/cardiovascular disease'},
 'cardiovascular medicine': {'cardiovascular medicine'},
 'cardiovascular surgery': {'cardiovascular surgery',
  'surgery - cardiovascular',
  'surgery, cardio

In [58]:
df['CLAIM_SPECIALTY_final'] = df['CLAIM_SPECIALTY_formatted']\
.apply(lambda x: list(paired[x])[0])

df['pairednum'] = df['CLAIM_SPECIALTY_formatted']\
.apply(lambda x: len(list(paired[x])))

In [59]:
print("Unique Claim Specialty: ", len(df['CLAIM_SPECIALTY_final'].unique()))

Unique Claim Specialty:  689


Since we need to manually check claims specialty groups we will focus only those which have the highest paid amount

In [60]:
maxamount = df.groupby(['CLAIM_SPECIALTY_final']).\
agg({'PAID_AMOUNT':'sum', 'pairednum': 'min'}).\
reset_index().\
sort_values(['PAID_AMOUNT'], ascending = False)

In [61]:
maxamount[0:20]

,CLAIM_SPECIALTY_final,PAID_AMOUNT,pairednum
244,inp,41412982,1
228,hospital,40080688,1
396,out,14024841,1
58,cardiology,7520528,2
208,hematology/oncology,5910652,3
245,internal medicine,4822104,1
285,medical doctor,4556985,1
603,snf,4248390,1
18,ambulance,2437935,1
173,family practice,2123451,1


### Manually checked top 20 - mapping is right

## As it comes from data description - we should have unique combinations of (Month, SERVICE_CATEGORY, CLAIM_SPECIALTY, Payer)

In [62]:
df.columns

Index(['MONTH', 'SERVICE_CATEGORY', 'CLAIM_SPECIALTY', 'PAYER', 'PAID_AMOUNT',
       'CLAIM_SPECIALTY_formatted', 'CLAIM_SPECIALTY_final', 'pairednum'],
      dtype='object')

In [63]:
np.max(df.groupby(['MONTH', 'SERVICE_CATEGORY', 'CLAIM_SPECIALTY', 'PAYER']).count().reset_index()['PAID_AMOUNT'])

1

In [64]:
np.max(df.groupby(['MONTH', 'SERVICE_CATEGORY', 'CLAIM_SPECIALTY_formatted', 'PAYER']).count().reset_index()['PAID_AMOUNT'])

2

We will deal with new aggregation logic on the analytics stage

# Found that we have 2 bagged months - 202006 is empty and 2019 has 13 month - I will drop 201900

In [65]:
df = df[(df.MONTH != 202007) & (df.MONTH != 201900)]

# Save results

In [66]:
df.to_csv('claims_sample_data_cleared.csv')